Erin Orbits

For this homework, you are a data scientist working for Pronto (before the end of their contract with the City of Seattle). Your job is to assist in determining how to do end-of-day adjustments in the number of bikes at stations so that all stations will have enough bikes for the next day of operation (as estimated by the weekday average for the station for the year). Your assistance will help in constructing a plan for each day of the week that specifies how many bikes should be moved from each station and how many bikes must be delievered to each station.

Your assignment is to construct plots of the differences between 'from' and 'to' counts for each station by day of the week. Do this as a set of 7 subplots. You should use at least one function to construct your plots.


In [28]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
# The following ensures that the plots are in the notebook
%matplotlib inline

In [30]:
df_original = pd.read_csv("2015_trip_data.csv")
df_original.head(5)

,trip_id,starttime,stoptime,bikeid,tripduration,from_station_name,to_station_name,from_station_id,to_station_id,usertype,gender,birthyear
0,431,10/13/2014 10:31,10/13/2014 10:48,SEA00298,985.935,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Annual Member,Male,1960.0
1,432,10/13/2014 10:32,10/13/2014 10:48,SEA00195,926.375,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Annual Member,Male,1970.0
2,433,10/13/2014 10:33,10/13/2014 10:48,SEA00486,883.831,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Annual Member,Female,1988.0
3,434,10/13/2014 10:34,10/13/2014 10:48,SEA00333,865.937,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Annual Member,Female,1977.0
4,435,10/13/2014 10:34,10/13/2014 10:49,SEA00202,923.923,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Annual Member,Male,1971.0


#### First, need to create four new columns with the days of the week and dates extracted from the starttime and stoptime for each row.

Note: Not sure whether I should delete the rows with "Pronto shop" from the "original data frame," but given the goal and the fact that there were only a few mentions in the data, I decided to delete the rows with the "Pronto shop."

In [32]:
df_original = df_original[df_original.from_station_name != "Pronto shop"]
df_original = df_original[df_original.to_station_name != "Pronto shop"]

Add "day of week" columns to the original data frame to correspond with the "To" and "From" groups using the days of the week from the starttime and stoptime columns.

In [33]:
df_original["from_day_of_week"] = pd.DatetimeIndex(df_original['starttime']).dayofweek
df_original["to_day_of_week"] = pd.DatetimeIndex(df_original['stoptime']).dayofweek

Add the dates columns to correspond with the "To" and "From" groups using the dates from the starttime and stoptime columns.

In [34]:
df_original["from_date"] = pd.DatetimeIndex(df_original['starttime']).date
df_original["to_date"] = pd.DatetimeIndex(df_original['stoptime']).date

Check that the four new columns were added to the data frame.

In [ ]:
df_original.columns

#### Second, I counted the number of bikes going to each station and from each station on each day of the week. 
With groupby you can group by multiple values and use aggregation functions like mean, median, sum, minimum, maximum, standard deviation, or count: <br>
>  `<data object>.groupby(<grouping values>).<aggregate>()` 

In [48]:
# Create a groupby variable that groups stations by day of week and counts the days of the week
# example: data.groupby([times.hour, 'gender'])['tripminutes'].mean()
toStation_day_count = df_original.groupby(["to_station_id", "to_day_of_week"])["to_day_of_week"].count()
toStation_day_count.head(5)

to_station_id  to_day_of_week
BT-01          0                 791
               1                 675
               2                 696
               3                 791
               4                 853
Name: to_day_of_week, dtype: int64

In [49]:
fromStation_day_count = df_original.groupby(["from_station_id", "from_day_of_week"])["from_day_of_week"].count()
fromStation_day_count.head(5)

from_station_id  from_day_of_week
BT-01            0                   806
                 1                   724
                 2                   679
                 3                   801
                 4                   921
Name: from_day_of_week, dtype: int64

Tried to count the number of weekdays, but couldn't figure it out... Things more or less went off the rails at this point...

In [70]:
# numpy.unique(array, return_index=False, return_inverse=False, return_counts=False) 
# Returns sorted unique elements of an array w/ 3 optional outputs in addition to the unique elements: 
# the indices of the input array that give the unique values, the indices of the unique array that 
# reconstruct the input array, and the number of times each unique value comes up in the input array.
from_x = df_original.groupby(["from_date", "from_day_of_week"])["from_day_of_week"].count()
from_x_index = from_x.index
from_x_index[:][:]
np.unique(df_original["from_date"])
(df_original["from_date"].count())/7
# num_from_weekdays = np.unique(df_original["from_day_of_week"], return_counts=True)
# (array([0, 1, 2, 3, 4, 5, 6]),
#  array([21262, 20462, 20746, 21502, 21095, 20358, 17407], dtype=int64))



20404.571428571428

In [ ]:
num_to_weekdays = np.unique(df_original["to_day_of_week"], return_counts=True)
# (array([0, 1, 2, 3, 4, 5, 6]),
#  array([21258, 20478, 20729, 21496, 21069, 20348, 17454], dtype=int64))

With value_counts, the Pandas method takes a series object and returns object containing counts of unique values: <br>
> series.value_counts(normalize=False, sort=True, ascending=False, bins=None, dropna=True)

In [72]:
# pandas.get_dummies(data, prefix=None, prefix_sep='_', dummy_na=False, columns=None, sparse=False, drop_first=False)

# pd.value_counts(to_station_count["day_of_week"], sort=True)
# to_mean = df_original.groupby(["day_of_week", "to_station_id"])["day_of_week"].mean()
to_station = df_original.groupby(["to_day_of_week", "to_station_id"])["to_day_of_week"]
to_counts = df_original["to_station_id"].value_counts(normalize=True, sort=True, dropna=True)
to_counts.head(10)
np.unique(df_original["to_station_id"])
per_day_station_count = df_original.groupby(["to_date", "to_day_of_week", "to_station_id"])["to_date"].count()
per_day_station_count.head(5)

to_date     to_day_of_week  to_station_id
2014-10-13  0               BT-01             6
                            BT-03            11
                            BT-04             3
                            BT-05             6
                            CBD-03           16
Name: to_date, dtype: int64

In [75]:
##### Scipy Stats #####
# scipy.stats.binned_statistic(x, values, statistic='mean', bins=10)
# scipy.stats.binned_statistic_2d(x, y, values, statistic='mean', bins=10, range=None, expand_binnumbers=False)
# scipy.stats.binned_statistic_2d([0,1,2,3,4,5,6], 
#                                ['BT-01', 'BT-03', 'BT-04', 'BT-05', 'CBD-03', 'CBD-04', 'CBD-05'])
monday_df = df_original[df_original["to_day_of_week"] ==0] # 21258 rows × 16 columns
monday_df.index

Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            142836, 142837, 142838, 142839, 142840, 142841, 142842, 142843,
            142844, 142845],
           dtype='int64', length=21258)

In [77]:
to_index = to_station_count.index  # names=['stop_date', 'day_of_stop_week', 'to_station_id'])
to_index.levshape # (365, 7, 54)

(365, 7, 54)

In [ ]:
# Need to compute the average value across the days for each station. The groupby produced a MultiIndex. 
# So, further operations on the result must take this into account.
# Example:  h_index = groupby_day_from.index
#           h_index.levshape  # Size of the components of the MultiIndex

# from_station_count is a Series pandas object with a multilevel index (so it has no columns)
from_station_count = df_original.groupby(["from_date", "from_day_of_week", "from_station_id"])["from_day_of_week"].count()
from_index = from_station_count.index
from_index.levshape # (365, 7, 54)


In [79]:
from_means = from_station_count.groupby(level=[0,1,2]).mean() # same as from_station_groupby values
from_station_count.index.levels
from_means

start_date  day_of_start_week  from_station_id
2014-10-13  0                  BT-01              10
                               BT-03               6
                               BT-04              13
                               BT-05               1
                               CBD-03             14
                               CBD-05             18
                               CBD-06             26
                               CBD-07              6
                               CBD-13              8
                               CH-01               3
                               CH-02              21
                               CH-03               1
                               CH-05              11
                               CH-06               6
                               CH-07               7
                               CH-08              12
                               CH-09               5
                               CH-12               2

FrozenList([[2014-10-13, 2014-10-14, 2014-10-15, 2014-10-16, 2014-10-17, 2014-10-18, 2014-10-19, 2014-10-20, 2014-10-21, 2014-10-22, 2014-10-23, 2014-10-24, 2014-10-25, 2014-10-26, 2014-10-27, 2014-10-28, 2014-10-29, 2014-10-30, 2014-10-31, 2014-11-01, 2014-11-02, 2014-11-03, 2014-11-04, 2014-11-05, 2014-11-06, 2014-11-07, 2014-11-08, 2014-11-09, 2014-11-10, 2014-11-11, 2014-11-12, 2014-11-13, 2014-11-14, 2014-11-15, 2014-11-16, 2014-11-17, 2014-11-18, 2014-11-19, 2014-11-20, 2014-11-21, 2014-11-22, 2014-11-23, 2014-11-24, 2014-11-25, 2014-11-26, 2014-11-27, 2014-11-28, 2014-11-29, 2014-11-30, 2014-12-01, 2014-12-02, 2014-12-03, 2014-12-04, 2014-12-05, 2014-12-06, 2014-12-07, 2014-12-08, 2014-12-09, 2014-12-10, 2014-12-11, 2014-12-12, 2014-12-13, 2014-12-14, 2014-12-15, 2014-12-16, 2014-12-17, 2014-12-18, 2014-12-19, 2014-12-20, 2014-12-21, 2014-12-22, 2014-12-23, 2014-12-24, 2014-12-25, 2014-12-26, 2014-12-27, 2014-12-28, 2014-12-29, 2014-12-30, 2014-12-31, 2015-01-01, 2015-01-02, 201

In [ ]:
# create a dataframe
# group_df = {"day_of_week": to_station_groupby["day_of_week"], "station_id":to_station_groupby["to_station_id"]}
df_counts = pd.DataFrame({'From': from_station_count.sort_index(), 'To': to_station_count.sort_index()})
df_counts.index.get_level_values(0)

In [ ]:
df_counts.index.levshape # get length of indices
df_counts.index.names # gives names of indices, initially set to ['day_of_week', 'from_station_id']
df_counts.index = df_counts.rename(index=str, columns={"from_day_of_week": "from_day_of_week", "from_station_id": "station"})
df_counts.head(10) # not sure why this didn't change the names of the indices

In [ ]:
df_counts.index.levels[0] # get days of week index Int64Index([0, 1, 2, 3, 4, 5, 6], dtype='int64')
df_counts.info()

In [ ]:
df_counts.index.levels[1]

Add the Delta column to the df_counts data frame to calculate the difference between the bikes going to stations and the bikes leaving from stations each day of the week.

In [ ]:
# Delta column gives # bikes at station at end of day, Delta = To - From 
df_counts["Delta"] = df_counts['To'] - df_counts['From']
df_counts.head(10)

In [ ]:
df_counts.index
len(df_counts.index)
len(df_counts.index.levels[1])

In [ ]:
"""
Plotting two variables as a bar chart
"""
n_groups = len(df_counts.index.levels[1])
index = np.arange(n_groups)  # The "raw" x-axis of the bar plot
fig = plt.figure(figsize=(16, 8))  # Controls global properties of the bar plot

bar_width = 0.35  # Width of the bars
opacity = 0.6  # How transparent the bars are
rects1 = plt.bar(index, df_counts.From, bar_width,
                 alpha=opacity,
                 color='b',
                 label='From')
rects2 = plt.bar(index + bar_width, df_counts.To, bar_width,
                 alpha=opacity,
                 color='r',
                 label='To')
plt.xticks(index + bar_width / 2, df_counts.index)
_, labels = plt.xticks()  # Get the new labels of the plot
plt.setp(labels, rotation=90)  # Rotate labels to make them readable
plt.legend()

plt.xlabel('Station')
plt.ylabel('Counts')
plt.title('Station Counts')
plt.show()

In [ ]:
def plot_bar1(df, column, opts):
    """
    Does a bar plot for a single column.
    :param pd.DataFrame df:
    :param str column: name of the column to plot
    :param dict opts: key is plot attribute
    """
    n_groups = len(df.index)
    index = np.arange(n_groups)  # The "raw" x-axis of the bar plot
    rects1 = plt.bar(index, df[column])
    if 'xlabel' in opts:
      plt.xlabel(opts['xlabel'])
    else:
      labels = " "
    if 'ylabel' in opts:
      plt.ylabel(opts['ylabel'])
      plt.legend(opts['ylabel'])
    if ('xticks' and 'xticks') in opts:
      plt.xticks(index, df.index)  # Convert "raw" x-axis into labels
      _, labels = plt.xticks()  # Get the new labels of the plot
      plt.setp(labels, rotation=90)  # Rotate labels to make them readable
    else:
      labels = ['' for x in df.index]
      plt.xticks(index, labels)   
    if 'ylim' in opts:
      plt.ylim(opts['ylim'])
    if 'title' in opts:
      plt.title(opts['title'])
    

In [ ]:
def plotNbar(df, columns, opts):
    """
    Does a bar plot for a single column.
    :param pd.DataFrame df:
    :param list-of-str columns: names of the column to plot
    :param dict opts: key is plot attribute
    """
    num_columns = len(columns)
    local_opts = dict(opts)  # Make a deep copy of the object
    idx = 0
    for column in columns:
        idx += 1
        local_opts['xticks'] = False
        local_opts['xlabel'] = ''
        if idx == num_columns:
          local_opts['xticks'] = True
          local_opts['xlabel'] = opts['xlabel']
        plt.subplot(num_columns, 1, idx)
        # calls plot_bar1 to plot each column
        plot_bar1(df, column, local_opts)


In [ ]:
fig = plt.figure(figsize=(16, 8))  # Controls global properties of the bar plot
opts = {'title': 'Station Counts','xlabel': 'Stations', 'ylabel': 'Daily Counts', 'ylim': [0, 1500]}
plotNbar(df_counts, ['To', 'From'], opts)

In [ ]:
###### Indexing and Selecting Data With Pandas ######
# .iloc[row,column]
# Select all rows by index label, 
# Example: Select all rows with index label "Arizona"
#  df.loc[:'Arizona']
# .ix is the combination of both .loc and .iloc. Integers are first considered labels, 
# but if not found, falls back on positional indexing
Monday = df_counts.ix[0]
Tuesday = df_counts.ix[1]
Tuesday.head(10)
type(Monday)

In [ ]:
fig = plt.figure(figsize=(16, 8))  # Controls global properties of the bar plot
opts = {'title': 'Station Counts','xlabel': 'Stations', 'ylabel': 'Daily Counts', 'ylim': [-600, 400]}
plotNbar(Monday,["Delta"], opts)

In [ ]:
def plotN(df, groups, columns, opts):
    """
    Constructs a plot for each group, each plot is constructed by calling plotNbar to draw a bar plot for a each column.
    :param pd.DataFrame df:
    :param df index groups: index levels for each plot
    :param list-of-str columns: names of the column to plot
    :param dict opts: key is plot attribute
    """
    num_groups = len(groups)
    idx = 0
    for group in groups:
        columns = df.ix[idx]
        idx += 1
        plotNbar(df, columns, opts)
    

In [ ]:
plotN(df_counts, df_counts.index.levels[0], df_counts[,0:1], opts)

In [ ]:
df_counts.index.levels[0][1]
df_counts.index.labels[0][0]
gb = df_counts.groupby(level=0)
gb.groups[0]
df_counts.

In [ ]:
df_counts.index.levels[0]

In [ ]:
days = ("Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday")
for i in len(days):
    days[i] = df_counts.groupby(level=[i])
    i += 1
days

In [ ]:
Monday = df_counts.index.levels[0][0]
df_counts.xs(0, level=0, drop_level=False)
df_counts.xs(1, level=0, drop_level=False)


In [ ]:
import seaborn

g = seaborn.FacetGrid(data=df_counts, col=df_counts.index.levels[0], col_wrap)
g.map(seaborn.distplot, "Delta")
seaborn.plt.show()